In [1]:
import torch
from torch import nn
from d2l import torch as d2l
import torchvision
from torch.utils.data import DataLoader
import torchsummary
import time

/home/whr-pc-ubuntu/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/whr-pc-ubuntu/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


# 加载数据

MNIST

In [2]:
batch_size = 256
seed = 0

In [3]:
def set_seed(seed):
    torch.manual_seed(seed)

In [4]:
def load_data(batch_size):
    dataset_path = "/home/whr-pc-ubuntu/code/dataset"
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        ])
    train_dataset = torchvision.datasets.MNIST(dataset_path,True,transform,download=True)
    test_dataset = torchvision.datasets.MNIST(dataset_path,False,transform,download=True)

    train_dataloader = DataLoader(train_dataset,batch_size,shuffle=True)
    test_dataloader = DataLoader(test_dataset,batch_size,shuffle=True)

    return train_dataloader,test_dataloader

In [5]:
train_dataloader,test_dataloader = load_data(batch_size)

In [6]:
# 获得batch时再设置seed，否则每次训练时seed还是不一样
set_seed(0)
next(iter(train_dataloader))[1]


tensor([6, 8, 8, 7, 8, 0, 0, 5, 6, 0, 0, 3, 5, 8, 1, 6, 2, 4, 1, 4, 9, 4, 3, 4,
        1, 4, 8, 7, 6, 3, 1, 8, 4, 4, 6, 2, 4, 2, 9, 5, 7, 8, 0, 7, 5, 5, 9, 3,
        6, 6, 4, 6, 7, 8, 3, 6, 2, 2, 7, 7, 2, 4, 4, 9, 9, 5, 2, 2, 9, 0, 9, 4,
        6, 5, 8, 5, 7, 1, 4, 2, 1, 8, 8, 6, 9, 7, 8, 0, 0, 8, 8, 6, 5, 0, 6, 9,
        7, 4, 1, 8, 1, 4, 0, 3, 5, 4, 6, 8, 1, 8, 7, 5, 5, 6, 0, 6, 5, 4, 9, 2,
        6, 0, 3, 1, 5, 8, 4, 0, 0, 9, 2, 5, 1, 1, 1, 8, 7, 7, 8, 0, 9, 2, 6, 4,
        3, 9, 0, 5, 4, 5, 9, 8, 4, 8, 4, 6, 3, 1, 2, 3, 2, 6, 5, 2, 1, 8, 7, 3,
        0, 8, 5, 6, 2, 2, 1, 8, 6, 3, 9, 5, 6, 9, 0, 2, 0, 9, 7, 5, 9, 4, 7, 2,
        6, 0, 3, 8, 7, 5, 8, 9, 6, 3, 4, 5, 9, 4, 8, 1, 7, 9, 9, 0, 4, 8, 7, 7,
        0, 3, 9, 3, 3, 8, 1, 7, 7, 9, 9, 1, 1, 8, 0, 3, 3, 7, 3, 5, 1, 6, 8, 9,
        8, 3, 5, 0, 5, 6, 7, 6, 7, 1, 3, 1, 7, 1, 4, 3])

# 网络结构

In [7]:
class FedAvgCNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv2d_1 = nn.Conv2d(1,32,(5,5))
        self.max_pool = nn.MaxPool2d((2,2))
        self.conv2d_2 = nn.Conv2d(32,64,(5,5))
        self.flat = nn.Flatten()
        self.fc = nn.Linear(1024,512)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        self.linear = nn.Linear(512,10)

    def forward(self,x):
        start = time.time()
        x = self.conv2d_1(x) # 32,24,24
        end = time.time()
        print('conv2d_1',end-start)
        
        x = self.relu(x) # 32,24,24
        x = self.max_pool(x) # 32,12,12

        start = time.time()
        x = self.conv2d_2(x) # 32,12,12
        end = time.time()
        print('conv2d_2',end-start)
        
        x = self.relu(x) # 32,12,12
        x = self.max_pool(x) # 32,6,6


        x = self.flat(x) # 1152

        start = time.time()
        x = self.fc(x) # 512
        end = time.time()
        print('fc',end-start)
        
        x = self.relu(x) # 512
        x = nn.Dropout(0.5)(x) # 512

        x = self.softmax(x) # 512

        x = self.linear(x) # 10

        return x

    # def forward(self,x):
    #     x = self.conv2d_1(x) # 32,24,24
    #     print(x.shape)

    #     x = self.relu(x) # 32,24,24
    #     print(x.shape)
        
    #     x = self.max_pool(x) # 32,12,12
    #     print(x.shape)

    #     x = self.conv2d_2(x) # 64,8,8
    #     print(x.shape)

    #     x = self.relu(x) # 64,8,8
    #     print(x.shape)

    #     x = self.max_pool(x) # 64,4,4
    #     print(x.shape)

    #     x = self.flat(x) # 1024
    #     print(x.shape)

    #     x = self.fc(x) # 512
    #     print(x.shape)

    #     x = self.relu(x) # 512
    #     print(x.shape)

    #     x = nn.Dropout(0.5)(x) # 512
    #     print(x.shape)

    #     x = self.softmax(x) # 512
    #     print(x.shape)

    #     x = self.linear(x) # 10
    #     print(x.shape)

    #     return x

## CPU

In [8]:
cnn = FedAvgCNN()

In [13]:
cnn(next(iter(train_dataloader))[0])

conv2d_1 0.0033109188079833984
conv2d_2 0.005147218704223633
fc 0.0010139942169189453


/tmp/ipykernel_358369/236861498.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x) # 512


tensor([[ 0.0031,  0.0266, -0.0212,  ..., -0.0124,  0.0224, -0.0129],
        [ 0.0032,  0.0266, -0.0211,  ..., -0.0124,  0.0224, -0.0129],
        [ 0.0031,  0.0266, -0.0212,  ..., -0.0126,  0.0223, -0.0127],
        ...,
        [ 0.0030,  0.0266, -0.0211,  ..., -0.0125,  0.0224, -0.0128],
        [ 0.0031,  0.0266, -0.0212,  ..., -0.0126,  0.0222, -0.0127],
        [ 0.0031,  0.0267, -0.0211,  ..., -0.0126,  0.0223, -0.0127]],
       grad_fn=<AddmmBackward0>)

## GPU

In [14]:
device = d2l.try_gpu()
device


device(type='cuda', index=0)

In [15]:
cnn = FedAvgCNN()
cnn = cnn.to(device)

In [16]:
x = next(iter(train_dataloader))[0].to(device)

In [20]:
cnn(x)

conv2d_1 0.0004572868347167969
conv2d_2 0.00012993812561035156
fc 8.463859558105469e-05


/tmp/ipykernel_358369/236861498.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x) # 512


tensor([[-0.0263, -0.0118,  0.0006,  ...,  0.0289,  0.0086,  0.0117],
        [-0.0260, -0.0118,  0.0005,  ...,  0.0287,  0.0087,  0.0118],
        [-0.0260, -0.0118,  0.0006,  ...,  0.0286,  0.0085,  0.0119],
        ...,
        [-0.0260, -0.0119,  0.0007,  ...,  0.0287,  0.0087,  0.0120],
        [-0.0263, -0.0118,  0.0007,  ...,  0.0287,  0.0086,  0.0119],
        [-0.0261, -0.0119,  0.0007,  ...,  0.0287,  0.0087,  0.0118]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [36]:
torchsummary.summary(cnn)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            832
├─MaxPool2d: 1-2                         --
├─Conv2d: 1-3                            51,264
├─Flatten: 1-4                           --
├─Linear: 1-5                            524,800
├─ReLU: 1-6                              --
├─Softmax: 1-7                           --
├─Linear: 1-8                            5,130
Total params: 582,026
Trainable params: 582,026
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            832
├─MaxPool2d: 1-2                         --
├─Conv2d: 1-3                            51,264
├─Flatten: 1-4                           --
├─Linear: 1-5                            524,800
├─ReLU: 1-6                              --
├─Softmax: 1-7                           --
├─Linear: 1-8                            5,130
Total params: 582,026
Trainable params: 582,026
Non-trainable params: 0